<a href="https://colab.research.google.com/github/sandeep-1215/Smart-Ambulance-Detection/blob/main/Copy_of_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Training of ambulance object detection in image and video

In [ ]:
# Install YOLOv8 and OpenCV
!pip install ultralytics opencv-python

# Import necessary modules
from google.colab import drive
import shutil
import os
import zipfile
import glob
from ultralytics import YOLO

# ✅ Step 1: Mount Google Drive
drive.mount('/content/drive')

# ✅ Step 2: Define Paths
ZIP_PATH = "/content/drive/MyDrive/ambulance.zip"  # Ensure correct path
MODEL_SAVE_PATH = "/content/drive/MyDrive/ambulance_detection_model.pt"
UNZIP_PATH = "/content"
DATASET_PATH = "/content/ambulance"

# ✅ Step 3: Extract Dataset (Every Run)
if os.path.exists(ZIP_PATH):
    print("🚀 Extracting dataset...")
    os.makedirs(UNZIP_PATH, exist_ok=True)
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(UNZIP_PATH)
    print("✅ Dataset extracted successfully!")
else:
    print("❌ Dataset not found in Drive! Please check the path.")
    exit()

# ✅ Step 4: Delete Old Training Runs Before Retraining
train_dir = "/content/runs/detect/train"
if os.path.exists(train_dir):
    print("🗑 Deleting old training runs...")
    shutil.rmtree(train_dir)

# ✅ Step 5: Validate Dataset (Check Image-Label Pairs)
image_files = glob.glob(f"{DATASET_PATH}/images/*/*.jpg") + glob.glob(f"{DATASET_PATH}/images/*/*.png")
label_files = glob.glob(f"{DATASET_PATH}/labels/*/*.txt")

# Ensure every image has a corresponding label
missing_labels = []
for img in image_files:
    label_path = img.replace("images", "labels").replace(".jpg", ".txt").replace(".png", ".txt")
    if not os.path.exists(label_path):
        missing_labels.append(img)

if missing_labels:
    print(f"❌ {len(missing_labels)} images missing labels! Check dataset.")
    exit()
else:
    print("✅ All images have corresponding labels!")

# ✅ Step 6: Check data.yaml for Proper Paths
yaml_path = f"{DATASET_PATH}/data.yaml"
if not os.path.exists(yaml_path):
    print("❌ data.yaml file is missing! Please check the dataset.")
    exit()
else:
    print("✅ data.yaml found!")

# ✅ Step 7: Train YOLOv8 Model (50 Epochs, GPU Optimized)
print("🚀 Training YOLOv8 Model...")
model = YOLO("yolov8m.pt")  # YOLOv8m for better accuracy

results = model.train(
    data=yaml_path,  # ✅ Ensure correct dataset path
    epochs=100,       # ✅ Increased to 50 epochs
    imgsz=640,       # ✅ Best balance for accuracy
    batch=8,         # ✅ Good batch size for GPU (RTX 3050)
    workers=2,       # ✅ Uses multiple workers for loading data
    device="cuda",   # ✅ Uses GPU
    save=True,       # ✅ Saves model checkpoints
    save_period=1,   # ✅ Saves after every epoch
    half=False       # ✅ Enables mixed precision for faster training
)

# ✅ Step 8: Save the Best Model
model_path = "/content/runs/detect/train/weights/best.pt"
if os.path.exists(model_path):
    print("✅ Training complete! Saving model to Google Drive...")
    shutil.move(model_path, MODEL_SAVE_PATH)  # Save best model
else:
    print("❌ Training failed! Model file not found.")
    exit()


  Using cached ultralytics-8.3.99-py3-none-any.whl.metadata (37 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.m

100%|██████████| 49.7M/49.7M [00:00<00:00, 314MB/s]


Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/ambulance/data.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=1, cache=False, device=cuda, workers=2, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

100%|██████████| 755k/755k [00:00<00:00, 89.9MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

...

    100/100      8.85G     0.3344     0.2178      1.014          2        640: 100%|██████████| 114/114 [00:31<00:00,  3.61it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:02<00:00,  3.51it/s]

                   all        143        148      0.928      0.917      0.963      0.801



100 epochs completed in 1.054 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,842,076 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:04<00:00,  2.10it/s]


                   all        143        148      0.939      0.955      0.972      0.811
             Ambulance         58         60          1      0.959      0.994       0.86
                   Bus         38         40      0.999       0.95      0.984        0.8
                   Car         24         25      0.872       0.92      0.949      0.793
                 Truck         23         23      0.884      0.993      0.962      0.792
Speed: 0.7ms preprocess, 12.1ms inference, 0.0ms loss, 4.6ms postprocess per image
Results saved to runs/detect/train
✅ Training complete! Saving model to Google Drive...


In [ ]:
# Training for Siren Detection

In [ ]:
# Import necessary libraries
!pip install gradio pydub scikit-learn matplotlib

import joblib
import numpy as np
import gradio as gr
import os
import librosa
import soundfile as sf
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from pydub import AudioSegment  # For audio file manipulation

# Function to extract MFCC features from an audio file
def extract_features(audio_path):
    y, sr = librosa.load(audio_path, sr=None)  # Load audio file with its sample rate
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # Extract 13 MFCC features
    return np.mean(mfccs, axis=1)  # Compute mean across time axis to get feature vector

# Initialize lists to store features and labels
X, y = [], []

# ✅ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ Path to the dataset stored in Google Drive
dataset_path = "/content/drive/MyDrive/sound_files"

# 🚑 Load ambulance siren sounds (Label = 1)
for file in os.listdir(os.path.join(dataset_path, "ambulance")):
    audio_path = os.path.join(dataset_path, "ambulance", file)
    features = extract_features(audio_path)
    if features is not None:
        X.append(features)
        y.append(1)  # Assign label 1 (siren)

# 🚗🚒 Load non-siren sounds (firetruck, traffic) (Label = 0)
for category in ["firetruck", "traffic"]:
    folder_path = os.path.join(dataset_path, category)
    for file in os.listdir(folder_path):
        audio_path = os.path.join(folder_path, file)
        features = extract_features(audio_path)
        if features is not None:
            X.append(features)
            y.append(0)  # Assign label 0 (non-siren)

# ✅ Convert feature and label lists into NumPy arrays for training
X = np.array(X)
y = np.array(y)

# ✅ Normalize features using StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# ✅ Train an SVM classifier
classifier = SVC(kernel="linear")  # Use linear kernel for binary classification
classifier.fit(X, y)  # Train the model on the dataset
print("✅ Model trained successfully!")

# ✅ Define paths to save the model in Google Drive
model_save_path = "/content/drive/MyDrive/siren_model.pkl"
scaler_save_path = "/content/drive/MyDrive/scaler.pkl"

# ✅ Save the trained model and scaler for future use
joblib.dump(classifier, model_save_path)  # Save the trained SVM model
joblib.dump(scaler, scaler_save_path)  # Save the scaler used for feature normalization

print(f"✅ Siren Model saved at: {model_save_path}")
print(f"✅ Scaler saved at: {scaler_save_path}")


Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


✅ Model trained successfully!
✅ Siren Model saved at: /content/drive/MyDrive/siren_model.pkl
✅ Scaler saved at: /content/drive/MyDrive/scaler.pkl


In [ ]:
# Testing using new data

In [ ]:
!pip install gradio torch torchvision torchaudio ultralytics opencv-python numpy joblib librosa moviepy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.4 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import torch
import os
import time
import cv2
import numpy as np
import shutil
import joblib
import librosa
from ultralytics import YOLO
from moviepy.editor import VideoFileClip, AudioFileClip

# Load YOLO model for ambulance detection
MODEL_PATH = "/content/drive/MyDrive/ambulance_detection_model.pt"
model = YOLO(MODEL_PATH)

# Load trained SVM model for siren detection
svm_model = joblib.load("/content/drive/MyDrive/siren_model.pkl")
scaler = joblib.load("/content/drive/MyDrive/scaler.pkl")

def extract_audio_features(audio_path):
    y, sr = librosa.load(audio_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_mean = np.mean(mfccs, axis=1)
    return scaler.transform([mfccs_mean])

def detect_siren(audio_path):
    features = extract_audio_features(audio_path)
    prediction = svm_model.predict(features)
    return prediction[0] == 1  # Returns True if siren is detected

def process_image(image, audio):
    # Save temporary files
    temp_image_path = "/content/temp_image.jpg"
    temp_audio_path = "/content/temp_audio.wav"
    image.save(temp_image_path)
    shutil.copy(audio, temp_audio_path)

    # Use original file name if available; otherwise, use a default name.
    if hasattr(image, "filename") and image.filename:
        original_image_name = os.path.basename(image.filename)
    else:
        original_image_name = "result_image.jpg"
    # This file will store the processed image (with YOLO detections)
    result_image_path = f"/content/drive/MyDrive/processed_{original_image_name}"

    # Dummy progress for processing
    for percent in range(1, 101, 5):
        yield percent, f"{percent}% - Processing...", None, None, None
        time.sleep(0.1)

    # Run YOLO model on the saved image and save the processed output
    results = model(temp_image_path)
    for result in results:
        result.save(result_image_path)

    for percent in range(80, 100, 5):
        yield percent, f"{percent}% - Finalizing...", None, None, None
        time.sleep(0.1)

    ambulance_detected = any(model.names[int(cls)] == "Ambulance"
                             for cls in results[0].boxes.cls.cpu().numpy())
    siren_detected = detect_siren(temp_audio_path)

    if ambulance_detected and siren_detected:
        status = "Ambulance and siren are detected\nAMBULANCE IS ON YOUR WAY !"
    elif ambulance_detected:
        status = "Ambulance detected but siren not detected"
    else:
        status = "Both ambulance and siren are not detected"

    # Return processed image path, not the original one.
    yield 100, "100% - Completed ✅", result_image_path, audio, status

def process_video_audio(video, audio, progress=gr.Progress()):
    if not video:
        return 0, "⚠ No video provided.", None, None, "⚠ No video provided."

    # Temporary file paths
    temp_video_path = "/content/temp_video.mp4"
    temp_audio_path = "/content/temp_audio.wav"
    shutil.copy(video, temp_video_path)
    shutil.copy(audio, temp_audio_path)

    # Use original video file name for saving processed result
    original_video_name = os.path.basename(video)
    output_path = f"/content/drive/MyDrive/processed_{original_video_name}"

    cap = cv2.VideoCapture(temp_video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames == 0:
        return 0, "⚠ Invalid or corrupted video file.", None, None, "⚠ Invalid or corrupted video file."

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    ambulance_detected = False
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model.predict(frame, conf=0.6)
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                cls = int(box.cls.item())
                label = model.names[cls]
                color = (0, 255, 0) if label == "Ambulance" else (0, 0, 255)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                if label == "Ambulance":
                    ambulance_detected = True

        out.write(frame)
        frame_count += 1
        percentage = int((frame_count / total_frames) * 100)
        progress(percentage / 100)
        time.sleep(0.05)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    siren_detected = detect_siren(temp_audio_path)
    if ambulance_detected and siren_detected:
        status = "Ambulance and siren are detected\nAMBULANCE IS ON YOUR WAY !"
    elif ambulance_detected:
        status = "Ambulance detected but siren not detected"
    else:
        status = "Both ambulance and siren are not detected"

    # Return the processed video file path
    return 100, "100% - Completed ✅", output_path, temp_audio_path, status

with gr.Blocks() as demo:
    gr.Markdown("# 🚑 Smart Ambulance Detection !")

    with gr.Tab("📷 Image + Audio"):
        img_input = gr.Image(type="pil")
        audio_input = gr.Audio(type="filepath")
        progress_bar_img = gr.Slider(minimum=0, maximum=100, step=1, label="Progress (%)", interactive=False)
        progress_text_img = gr.Text(label="Status", interactive=False)
        img_output = gr.Image()  # will display the processed image
        audio_output = gr.Audio()
        status_output = gr.Text()  # using gr.Text for plain text output

        img_btn = gr.Button("Detect")
        img_btn.click(process_image, inputs=[img_input, audio_input],
                      outputs=[progress_bar_img, progress_text_img, img_output, audio_output, status_output])

    with gr.Tab("🎥 Video + Audio"):
        video_input = gr.Video()
        audio_input_vid = gr.Audio(type="filepath")
        progress_bar_vid = gr.Slider(minimum=0, maximum=100, step=1, label="Progress (%)", interactive=False)
        progress_text_vid = gr.Text(label="Status", interactive=False)
        video_output = gr.Video()  # will display the processed video
        audio_output_vid = gr.Audio()
        status_output_vid = gr.Text()  # using gr.Text for plain text output

        video_btn = gr.Button("Detect")
        video_btn.click(process_video_audio, inputs=[video_input, audio_input_vid],
                        outputs=[progress_bar_vid, progress_text_vid, video_output, audio_output_vid, status_output_vid])

demo.launch()


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ambulance_detection_model.pt'

In [ ]:
# Install required packages
!pip install ultralytics opencv-python

In [ ]:
import gradio as gr
import cv2
from PIL import Image
from ultralytics import YOLO
from PIL import Image

# Load trained models
svm_model = joblib.load("siren_model.pkl")  # Load SVM model for siren detection
scaler = joblib.load("scaler.pkl")  # Load scaler for feature normalization
yolo_model = YOLO("/content/drive/MyDrive/ambulance_detection_model.pt")  # Load YOLO model for ambulance detection

# Function to extract features from an audio file for siren detection
def extract_features(audio_path):
    y, sr = librosa.load(audio_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfccs, axis=1)

# Function for siren detection
def detect_siren(audio_file):
    features = extract_features(audio_file)
    features = scaler.transform([features])  # Normalize features
    prediction = svm_model.predict(features)  # Predict using SVM
    return "🚨 Siren Detected!" if prediction[0] == 1 else "❌ No Siren Detected."

# Function for ambulance detection
def detect_ambulance(image_file):
    image = Image.open(image_file)  # Open image
    results = yolo_model(image)  # Perform YOLO object detection
    if len(results[0].boxes) > 0:
        return "✅ Ambulance Detected!"
    else:
        return "❌ No Ambulance Detected."

 # Function to detect ambulances in a video
from moviepy.editor import VideoFileClip
def detect_ambulance_in_video(video_path):
    try:
        clip = VideoFileClip(video_path)
        frame_count = 0
        detected_frames = 0

        for frame in clip.iter_frames(fps=30, dtype="uint8"):  # Extract frames at 30 FPS
            frame_count += 1
            frame_path = f"frame_{frame_count}.jpg"
            cv2.imwrite(frame_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

            result = yolo_model(frame_path)
            if len(result[0].boxes) > 0:
                detected_frames += 1  # Count detected frames

            os.remove(frame_path)  # Delete temp frame after processing

        clip.close()

        if detected_frames > 0:
            return f"✅ Ambulance detected in {detected_frames} frames!"
        else:
            return "❌ No ambulance detected in the video."

    except Exception as e:
        return f"⚠️ Error processing video: {str(e)}"


# Function to process input
def process_input(input_type, file):
    if input_type == "Image":
        return detect_ambulance(file.name)
    elif input_type == "Audio":
        return detect_siren(file.name)
    elif input_type == "Video":
        return detect_ambulance_in_video(file.name)
    else:
        return "⚠️ Invalid Input Type!"

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🚑 Ambulance and Siren Detection")

    input_type = gr.Radio(["Image", "Audio", "Video"], label="Choose Input Type", value="Image")
    file_input = gr.File(label="Upload File")

    output = gr.Textbox(label="Detection Output")

    submit_button = gr.Button("Detect")
    submit_button.click(process_input, inputs=[input_type, file_input], outputs=output)

# Launch interface
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8d5c385d11b3e74d37.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install ultralytics opencv-python moviepy pydub

image 1/1 /content/frame_42.jpg: 384x640 1 Ambulance, 2614.3ms
Speed: 3.5ms preprocess, 2614.3ms inference, 9.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/frame_43.jpg: 384x640 2 Ambulances, 4927.0ms
Speed: 4.6ms preprocess, 4927.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/frame_44.jpg: 384x640 1 Ambulance, 1 Car, 1 Truck, 3873.2ms
Speed: 26.3ms preprocess, 3873.2ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /content/frame_45.jpg: 384x640 (no detections), 3017.9ms
Speed: 8.6ms preprocess, 3017.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)



In [ ]:
#!pip install ultralytics opencv-python moviepy pydub
import cv2
import torch
import numpy as np
import librosa
import soundfile as sf
import os
import joblib
from ultralytics import YOLO
from moviepy.editor import VideoFileClip
from google.colab import files
from google.colab import drive

# ✅ Mount Google Drive
drive.mount('/content/drive')

# ✅ Load YOLO model for ambulance detection
model = YOLO("/content/drive/MyDrive/ambulance_detection_model.pt")  # Update with correct path
print("🚀 YOLO Model loaded successfully!")

# ✅ Load trained siren detection model & scaler from Google Drive
siren_model_path = "/content/drive/MyDrive/siren_model.pkl"
scaler_path = "/content/drive/MyDrive/scaler.pkl"

classifier = joblib.load(siren_model_path)
scaler = joblib.load(scaler_path)
print("🚀 Siren Model and Scaler loaded successfully!")

# ✅ Function to extract MFCC features for siren detection
def extract_features(audio_path):
    y, sr = librosa.load(audio_path, sr=None)  # Load audio file with its sample rate
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # Extract 13 MFCC features
    return np.mean(mfccs, axis=1)  # Compute mean across time axis to get feature vector

# ✅ Function to detect siren using the trained model
def detect_siren(audio_path):
    try:
        features = extract_features(audio_path)
        features = features.reshape(1, -1)  # Reshape for prediction
        features = scaler.transform(features)  # Normalize with the saved scaler
        prediction = classifier.predict(features)
        return prediction[0] == 1  # Returns True if siren is detected
    except Exception as e:
        print(f"❌ Siren detection error: {e}")
        return False

# ✅ Function to process images
def process_image(image_path, audio_path):
    img = cv2.imread(image_path)
    results = model.predict(img, conf=0.6)  # Run YOLO on image

    detected_classes = [model.names[box.cls.item()] for box in results[0].boxes]

    # Detect siren if ambulance is present
    siren_detected = detect_siren(audio_path) if "Ambulance" in detected_classes else False

    # Result logic
    if "Ambulance" in detected_classes:
        result_text = "🚑 Ambulance and Siren are detected!" if siren_detected else "🚑 Ambulance detected, but Siren not detected!"
    else:
        result_text = "❌ Both Ambulance and Siren are not detected!"

    # Get original file name and save processed image in Drive
    image_name = os.path.basename(image_path)
    processed_img_path = f"/content/drive/MyDrive/processed_{image_name}"
    cv2.imwrite(processed_img_path, cv2.cvtColor(results[0].plot(), cv2.COLOR_RGB2BGR))

    print(f"✅ Processed image saved in Google Drive: {processed_img_path}")
    print(result_text)
    return processed_img_path, result_text

# ✅ Function to process videos
def process_video(video_path, audio_path):
    clip = VideoFileClip(video_path)

    # Get original file name and save processed video in Drive
    video_name = os.path.basename(video_path)
    processed_video_path = f"/content/drive/MyDrive/processed_{video_name}"

    ambulance_detected = False
    output_frames = []

    for frame in clip.iter_frames(fps=10, dtype=np.uint8):
        results = model.predict(frame, conf=0.6)
        detected_classes = [model.names[box.cls.item()] for box in results[0].boxes]

        # Overlay detections on the frame
        processed_frame = results[0].plot()
        output_frames.append(processed_frame)

        if "Ambulance" in detected_classes:
            ambulance_detected = True  # No need to check further

    siren_detected = detect_siren(audio_path) if ambulance_detected else False

    # Result logic
    if ambulance_detected:
        result_text = "🚑 Ambulance and Siren are detected!" if siren_detected else "🚑 Ambulance detected, but Siren not detected!"
    else:
        result_text = "❌ Both Ambulance and Siren are not detected!"

    # Convert processed frames to video
    height, width, _ = output_frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(processed_video_path, fourcc, 10, (width, height))

    for frame in output_frames:
        out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    out.release()

    print(f"✅ Processed video saved in Google Drive: {processed_video_path}")
    print(result_text)
    return processed_video_path, result_text

# **✅ UPLOAD & DETECT LOGIC**
file_type = input("Upload Type (image+audio / video+audio): ").strip().lower()

if file_type == "image+audio":
    print("Upload Image")
    image_path = list(files.upload().keys())[0]

    print("Upload Audio")
    audio_path = list(files.upload().keys())[0]

    processed_image, result_text = process_image(image_path, audio_path)

elif file_type == "video+audio":
    print("Upload Video")
    video_path = list(files.upload().keys())[0]

    print("Upload Audio")
    audio_path = list(files.upload().keys())[0]

    processed_video, result_text = process_video(video_path, audio_path)

else:
    print("❌ Invalid input! Please enter 'image+audio' or 'video+audio'.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 YOLO Model loaded successfully!
🚀 Siren Model and Scaler loaded successfully!
Upload Type (image+audio / video+audio): video+audio
Upload Video


Saving istockphoto-543255860-640_adpp_is.mp4 to istockphoto-543255860-640_adpp_is.mp4
Upload Audio


Saving sound_7.wav to sound_7 (5).wav

0: 384x640 2 Cars, 1145.4ms
Speed: 3.8ms preprocess, 1145.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 969.9ms
Speed: 6.7ms preprocess, 969.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 947.3ms
Speed: 3.4ms preprocess, 947.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Car, 937.5ms
Speed: 4.3ms preprocess, 937.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 929.9ms
Speed: 10.5ms preprocess, 929.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 950.5ms
Speed: 4.4ms preprocess, 950.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 943.4ms
Speed: 3.4ms preprocess, 943.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 948.8ms
Speed: 3.7ms preprocess, 948.8ms inference, 1.0ms 